In [ ]:
!pip install pandas
!pip install gradio
!pip install langchain
!pip install sentence-transformers
!pip install -U langchain-community
!pip install requests
!pip install IPython
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import gradio as gr
from IPython.display import Image, display
import requests


In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HelpMate AI/FashionDB.csv')
df.head(2)



,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."


In [ ]:
# Fill missing values and create a combined description
df['description'] = df['description'].fillna('')
df['combined'] = df['name'] + ' ' + df['description']

# Select relevant columns
data = df[['p_id', 'combined', 'img']]

In [ ]:
df['img'].isnull().sum()

,img
0,http://assets.myntassets.com/assets/images/170...
1,http://assets.myntassets.com/assets/images/165...
2,http://assets.myntassets.com/assets/images/163...
3,http://assets.myntassets.com/assets/images/147...
4,http://assets.myntassets.com/assets/images/110...
...,...
14209,http://assets.myntassets.com/assets/images/154...
14210,http://assets.myntassets.com/assets/images/164...
14211,http://assets.myntassets.com/assets/images/163...
14212,http://assets.myntassets.com/assets/images/163...


In [ ]:
# Initialize embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a vector store
vector_store = FAISS.from_texts(data['combined'].tolist(), embedding_model)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Function to search for products based on user query and display images
def search_products(query, n_results=5):
    query_embedding = embedding_model.embed_query(query)
    results = vector_store.similarity_search_by_vector(query_embedding, k=n_results)

    # Get the indices from the results
    indices = [int(result.id) if result.id is not None else -1 for result in results]

    # Assuming 'image_url' column contains the image URLs
    image_urls = [data['img'].iloc[i] for i in indices if i != -1]

    # Display images
    for url in image_urls:
        try:
          display(Image(url=url))
        except Exception as e:
          print(f"Error displaying image from URL: {url}, Error: {e}")

    return [result.page_content for result in results]


In [ ]:
results = search_products("red dress", n_results=5)
for result in results:
    print(result)

Sera Red Two Piece Party Dress Red Two Piece Party Dress<br>Red foil print crop top has shoulder strap neck&nbsp;<br>Red foil print minin skirtThe model (height 5'8) is wearing a size S<p>Hand Wash,Flat Dry, Do Not Iron</p>
URBANIC Women Red Solid Straight Mini Skirt <p>Red solid straight mini skirt, has lace up , slip-on closure and straight hem</p><p>The model (height 5'8") is wearing size S</p><p>Polyester<br>Machine Wash</p>
Ginger By Lifestyle Magenta Sequin Embellished Skirt Magenta sequin embellished skirt, has an zip closure, front slit<br>The model (height 5'8") is wearing a size 28Material: Polyester<br>Dry clean
Dress My Angel Girls Red Printed Mini-Length Skorts Skirts <p>Red printed mini-length skorts skirt, has an elasticated waistband with slip-on closure, and straight hem</p><p>7-8Y</p><p>Polyester</p>
max Women Red Printed Skirts <p>Red printed  flared skirt, has an elasticated waistband, slip-on closure with drawstring , flared hem and taping border</p><p>100% RAYON<b

In [ ]:
# Create the Gradio interface
iface = gr.Interface(
    fn=search_products,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Textbox(lines=7, label="Search Results"),
    title="Product Search",
    description="Enter a query to search for products.",
)

# Launch the interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9087bff5b28eac4549.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
